# 5.6 Ordered sets

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Any set of numbers has a natural ordering, so numbers are often used to represent
entities, like time periods, whose ordering is essential to the specification of a model. To
describe the difference between this week's inventory and the previous week's inventory,
for example, we need the weeks to be ordered so that the "previous" week is always well
defined.

An AMPL model can also define its own ordering for any set of numbers or strings, by
adding the keyword `ordered` or `circular` to the set's declaration. The order in
which you give the set's members, in either the model or the data, is then the order in
which AMPL works with them. In a set declared `circular`, the first member is considered
to follow the last one, and the last to precede the first; in an `ordered` set, the first
member has no predecessor and the last member has no successor.

Ordered sets of strings often provide better documentation for a model's data than sets
of numbers. Returning to the multiperiod production model of [Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4), we observe
that there is no way to tell from the data which weeks the numbers 1 through `T` refer to, or
even that they are weeks instead of days or months. Suppose that instead we let the
weeks be represented by an ordered set that contains, say, `27sep`, `04oct`, `11oct`
and `18oct`. The declaration of `T` is replaced by
```
set WEEKS ordered;
```
and all subsequent occurrences of `1..T` are replaced by `WEEKS`. In the `Balance` constraint,
the expression `t-1` is replaced by `prev(t)`, which selects the member before `t`
in the set's ordering:
```
subject to Balance {p in PROD, t in WEEKS}:
  Make[p,t] + Inv[p,prev(t)] = Sell[p,t] + Inv[p,t]; # WRONG
```
This is not quite right, however, because when `t` is the first week in WEEKS, the member
`prev(t)` is not defined. When you try to solve the problem, you will get an error message
like this:
```
error processing constraint Balance['bands','27sep']:
       can't compute prev('27sep', WEEKS) --
	  '27sep' is the first member
```
One way to fix this is to give a separate balance constraint for the first period, in which
`Inv[p,prev(t)]` is replaced by the initial inventory, `inv0[p]`:
```
subject to Balance0 {p in PROD}:
  Make[p,first(WEEKS)] + inv0[p]
     = Sell[p,first(WEEKS)] + Inv[p,first(WEEKS)];
```
The regular balance constraint is limited to the remaining weeks:
```
subject to Balance {p in PROD, t in WEEKS: ord(t) > 1}:
  Make[p,t] + Inv[p,prev(t)] = Sell[p,t] + Inv[p,t];
```
The complete model and data are shown in Figures [5-3](../05/5_6_ordered_sets.ipynb#fig-5-3) and [5-4](../05/5_6_ordered_sets.ipynb#fig-5-4). As a tradeoff for more
meaningful week names, we have to write a slightly more complicated model.

As our example demonstrates, AMPL provides a variety of functions that apply specifically
to ordered sets. These functions are of three basic types.

<a id='fig-5-3'><center><b>Figure 5-3:</b> Production model with ordered sets (steelT2.mod).</center></a>

In [2]:
%%writefile steelT2.mod

set PROD;                                  # products
set WEEKS ordered;                         # number of weeks
param rate {PROD} > 0;           #    tons per hour produced
param inv0 {PROD} >= 0;          #    initial inventory
param avail {WEEKS} >= 0;        #    hours available in week
param market {PROD,WEEKS} >= 0;  #    limit on tons sold in week
param prodcost {PROD} >= 0;      # cost per ton produced
param invcost {PROD} >= 0;       # carrying cost/ton of inventory
param revenue {PROD,WEEKS} >= 0; # revenue/ton sold

var Make {PROD,WEEKS} >= 0;      # tons produced
var Inv {PROD,WEEKS} >= 0;       # tons inventoried
var Sell {p in PROD, t in WEEKS} >= 0, <= market[p,t]; # tons sold

maximize Total_Profit:
	sum {p in PROD, t in WEEKS} (revenue[p,t]*Sell[p,t] -
      prodcost[p]*Make[p,t] - invcost[p]*Inv[p,t]);
			   # Objective: total revenue less costs in all weeks

subject to Time {t in WEEKS}:
	sum {p in PROD} (1/rate[p]) * Make[p,t] <= avail[t];
			   # Total of hours used by all products
			   # may not exceed hours available, in each week

subject to Balance0 {p in PROD}:
	Make[p,first(WEEKS)] + inv0[p]
      = Sell[p,first(WEEKS)] + Inv[p,first(WEEKS)];

subject to Balance {p in PROD, t in WEEKS: ord(t) > 1}:
	Make[p,t] + Inv[p,prev(t)] = Sell[p,t] + Inv[p,t];
			   # Tons produced and taken from inventory
			   # must equal tons sold and put into inventory

Overwriting steelT2.mod


<a id='fig-5-4'><center><b>Figure 5-4:</b>  Data for production model.</center></a>

In [3]:
import pandas as pd

# Sets
PROD = ['bands', 'coils']
WEEKS = ['27sep', '04oct', '11oct', '18oct']

# Unified product parameters
df_prod = pd.DataFrame(
    [
        ['bands', 200, 10, 10, 2.5],
        ['coils', 140, 0, 11, 3.0]
    ],
    columns=['PROD', 'rate', 'inv0', 'prodcost', 'invcost']
).set_index('PROD')

# Time-based parameters
avail = pd.Series([40, 40, 32, 40], index=WEEKS, name='avail')

# Matrix parameters
revenue = pd.DataFrame(
    [
        [25, 26, 27, 27],
        [30, 35, 37, 39]
    ],
    index=PROD,
    columns=WEEKS
)

market = pd.DataFrame(
    [
        [6000, 6000, 4000, 6500],
        [4000, 2500, 3500, 4200]
    ],
    index=PROD,
    columns=WEEKS
)

In [4]:
ampl.read('steelT2.mod')

# Load sets
ampl.set['PROD'] = PROD
ampl.set['WEEKS'] = WEEKS

# Load parameter tables
ampl.set_data(df_prod, 'PROD')
ampl.param['avail'] = avail
ampl.param['revenue'] = revenue
ampl.param['market'] = market

ampl.solve(solver='highs')

HiGHS 1.11.0: optimal solution; objective 515033
16 simplex iterations
0 barrier iterations


First, there are functions that return a member from some absolute position in a set.
You can write `first(WEEKS)` and `last(WEEKS)` for the first and last members of
the ordered set `WEEKS`. To pick out other members, you can use `member(5,WEEKS)`,
say, for the 5th member of `WEEKS`. The arguments of these functions must evaluate to an
ordered set, except for the first argument of `member`, which can be any expression that
evaluates to a positive integer.

A second kind of function returns a member from a position relative to another member.
Thus you can write `prev(t,WEEKS)` for the member immediately before `t` in
`WEEKS`, and `next(t,WEEKS)` for the member immediately after. More generally,
expressions such as `prev(t,WEEKS,5)` and `next(t,WEEKS,3)` refer to the 5th
member before and the 3rd member after `t` in `WEEKS`. There are also "wraparound"
versions `prevw` and `nextw` that work the same except that they treat the end of the set
as wrapping around to the beginning; in effect, they treat all ordered sets as if their declarations were `circular`. In all of these functions, the first argument must evaluate to a
number or string, the second argument to an ordered set, and the third to an integer. Normally
the integer is positive, but zero and negative values are interpreted in a consistent
way; for instance, `next(t,WEEKS,0)` is the same as `t`, and `next(t,WEEKS,-5)` is
the same as `prev(t,WEEKS,5)`.

Finally, there are functions that return the position of a member within a set. The
expression `ord(t,WEEKS)` returns the numerical position of `t` within the set `WEEKS`,
or gives you an error message if `t` is not a member of `WEEKS`. The alternative
`ord0(t,WEEKS)` is the same except that it returns 0 if `t` is not a member of `WEEKS`.
For these functions the first argument must evaluate to a positive integer, and the second
to an ordered set.

If the first argument of `next`, `nextw`, `prev`, `prevw`, or `ord` is a dummy index that
runs over an ordered set, its associated indexing set is assumed if a set is not given as the
second argument. Thus in the constraint
```
subject to Balance {p in PROD, t in WEEKS: ord(t) > 1}:
    Make[p,t] + Inv[p,prev(t)] = Sell[p,t] + Inv[p,t];
```
the functions `ord(t)` and `prev(t)` are interpreted as if they had been written
`ord(t,WEEKS)` and `prev(t,WEEKS)`.

Ordered sets can also be used with any of the AMPL operators and functions that
apply to sets generally. The result of a `diff` operation preserves the ordering of the left
operand, so the material balance constraint in our example could be written:
```
subject to Balance {p in PROD, t in WEEKS diff {first(WEEKS)}}:
    Make[p,t] + Inv[p,prev(t)] = Sell[p,t] + Inv[p,t];
```
For `union`, `inter` and `symdiff`, however, the ordering of the result is not well
defined; AMPL treats the result as an unordered set.
<a id=p86></a>

For a set that is contained in an ordered set, AMPL provides a way to say that the
ordering should be inherited. Suppose for example that you want to try running the multiperiod
production model with horizons of different lengths. In the following declarations,
the ordered set `ALL_WEEKS` and the parameter `T` are given in the data, while the
subset `WEEKS` is defined by an indexing expression to include only the first `T` weeks:
```
set ALL_WEEKS ordered;
param T > 0 integer;
set WEEKS = {t in ALL_WEEKS: ord(t) <= T} ordered by ALL_WEEKS;
```
We specify ordered by `ALL_WEEKS` so that `WEEKS` becomes an ordered set, with its
members having the same ordering as they do in `ALL_WEEKS`. The `ordered by` and
`circular by` phrases have the same effect as the `within` phrase of [Section 5.4](../05/5_4_set_membership_operations_and_functions.ipynb)
together with `ordered` or `circular`, except that they also cause the declared set to
inherit the ordering from the containing set. There are also `ordered by reversed`
and `circular by reversed` phrases, which cause the declared set's ordering to be
the opposite of the containing set's ordering. All of these phrases may be used either
with a subset supplied in the data, or with a subset defined by an expression as in the
example above.

## Predefined sets and interval expressions

AMPL provides special names and expressions for certain common intervals and other
sets that are either infinite or potentially very large. Indexing expressions may not iterate
over these sets, but they can be convenient for specifying the conditional phrases in `set`
and `param` declarations.

AMPL intervals are sets containing all numbers between two bounds. There are intervals
of real (floating-point) numbers and of integers, introduced by the keywords
`interval` and `integer` respectively. They may be specified as closed, open, or halfopen,
following standard mathematical notation,

$$
\begin{align*}
\text{interval } [a, b] &\equiv \{ x : a \le x \le b \}, \\
\text{interval } (a, b] &\equiv \{ x : a < x \le b \}, \\
\text{interval } [a, b) &\equiv \{ x : a \le x < b \}, \\
\text{interval } (a, b) &\equiv \{ x : a < x < b \}, \\
\text{integer } [a, b] &\equiv \{ x \in \mathbb{Z} : a \le x \le b \}, \\
\text{integer } (a, b] &\equiv \{ x \in \mathbb{Z} : a < x \le b \}, \\
\text{integer } [a, b) &\equiv \{ x \in \mathbb{Z} : a \le x < b \}, \\
\text{integer } (a, b) &\equiv \{ x \in \mathbb{Z} : a < x < b \}
\end{align*}
$$

where `a` and `b` are any arithmetic expressions, and $\mathbb{Z}$ denotes the set of integers. In the
declaration phrases
```
in interval
within interval
ordered by [ reversed ] interval
circular by [ reversed ] interval
```

the keyword `interval` may be omitted.

As an example, in declaring [Chapter 1](../01/01.md)'s parameter `rate`, you can declare
```
param rate {PROD} in interval (0,maxrate];
```
to say that the production rates have to be greater than zero and not more than some previously
defined parameter `maxrate`; you could write the same thing more concisely as
```
param rate {PROD} in (0,maxrate];
```
or equivalently as
```
param rate {PROD} > 0, <= maxrate;
```
An open-ended interval can be specified by using the predefined AMPL parameter
`Infinity` as the right-hand bound, or `-Infinity` as the left-hand bound, so that
```
param rate {PROD} in (0,Infinity];
```
means exactly the same thing as
```
param rate {PROD} > 0;
```
in [Figure 1-4a](../01/tut_1_4.ipynb#fig-1-4a). In general, intervals do not let you say anything new in set or parameter
declarations; they just give you alternative ways to say things. (They have a more essential
role in defining imported functions, as discussed in Section A.22 xTODO.)

The predefined infinite sets `Reals` and `Integers` are the sets of all floating-point
numbers and integers, respectively, in numeric order. The predefined infinite sets
`ASCII`, `EBCDIC`, and `Display` all represent the universal set of strings and numbers
from which members of any one-dimensional set are drawn. `ASCII` and `EBCDIC` are
ordered by the `ASCII` and `EBCDIC` collating sequences, respectively. `Display` has the
ordering used in AMPL's `display` command (Section A.16 xTODO): numbers precede literals
and are ordered numerically; literals are sorted by the `ASCII` collating sequence.

As an example, you can declare
```
set PROD ordered by ASCII;
```
to make AMPL's ordering of the members of `PROD` alphabetical, regardless of their ordering
in the data. This reordering of the members of `PROD` has no effect on the solutions of
the model in [Figure 1-4a](../01/tut_1_4.ipynb#fig-1-4a), but it causes AMPL listings of most entities indexed over `PROD`
to appear in the same order (see A.6.2 xTODO).
